# Task 1

In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal

/Users/emilneby/miniforge3/envs/tkt4550/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Generate random vibration data for a shear frame of 8 storeys

Based on example notebook "00-generating-data.ipynb"

In [3]:
# Create a shear frame
sf = strid.utils.ShearFrame(8, 1e3, 1e7) #Defining 8 storeys here
sf.set_rayleigh_damping_matrix([sf.get_natural_frequency(1), sf.get_natural_frequency(sf.n)], [.05]*2)

# Determine the time discretization and period
Tmax = 1. / strid.w2f(sf.get_natural_frequency(1))
fmax = strid.w2f(sf.get_natural_frequency(sf.n))
T = 1000*Tmax
fs = 5 * fmax
t = np.arange(0., T, 1/fs)

# Define loads on system
## Unmeasureable: Stochastic loads on all floors (Process noise)
w = np.random.normal(size=(sf.n, t.size)) * 1e-1

## Load matrix, f
F = w.copy()

# Simulate response, accelerations at each floor measured
y0, _, _ = sf.simulate(t, F)

noise_std = y0.std()

# Add measurement noise
v = np.random.normal(size=y0.shape)*noise_std
y = y0 + v

plt.figure("Accelerations measured at top floor")
plt.plot(t, y[-1], label="w/noise")
plt.plot(t, y0[-1], label="wo/noise")
plt.legend()
plt.grid()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.figure("PSD of accelerations at top floor")
for yi in [y[-1], y0[-1]]:
    freqs, Gyy = scipy.signal.welch(yi, fs, nperseg=2**10)
    plt.semilogy(freqs, Gyy)


for n in range(1, 1+sf.n):
    plt.axvline(strid.w2f(sf.get_natural_frequency(n)), alpha=.3)
plt.ylabel('PSD')
plt.xlabel('Frequency (Hz)')
plt.grid()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
true_frequencies = np.array([sf.get_natural_frequency(i)/(2*np.pi) for i in range(1, sf.n+1)])
true_damping = np.array([sf.get_rayleigh_damping_ratio(i) for i in range(1, sf.n+1)])
true_modeshapes = np.array([sf.get_mode_shape(i) for i in range(1, sf.n+1)])

#Saving the data
np.savez("results/data-stochastic_8_floor.npz",
         y=y, fs=fs,
         true_frequencies=true_frequencies,
         true_damping=true_damping,
         true_modeshapes=true_modeshapes
        )

### Use covariance-driven stochastic subspace identification to perform system identification

Based on example notebook "01a-stochastic-system-identification-CovSSI.ipynb"

In [5]:
data = np.load("results/data-stochastic_8_floor.npz")
y = data["y"]
fs = data["fs"]
true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

In [6]:
ssid = strid.CovarianceDrivenStochasticSID(y, fs)

In [7]:
modes = {}
for i, order in enumerate(range(5, 50, 1)):
    A, C, G, R0 = ssid.perform(order, 25)
    modes[order] = strid.Mode.find_modes_from_ss(A, C, ssid.fs)

In [8]:
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

# Task 2
### Task 2.1 - finding relative difference

In [28]:
cluster_meat = []

for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))

    counter = 0
    for mode in modes_of_order:
        counter += 1
        #print("Mode " + str(counter) + "in order " + str(order))
        #print("Length of next inferior model order " + str(len(modes[order-1])))
        neighbour = strid.find_nearest_neighbour(mode, modes[order-1])

        mode.delta_frequency = strid.rel_diff_freq(neighbour.f, mode.f)
        mode.delta_damping = np.abs(neighbour.xi - mode.xi)
        mode.delta_mac = strid.utils.modal_assurance_criterion(neighbour.v, mode.v)

        cluster_meat.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        #np.append(cluster_meat, [mode.delta_frequency, mode.delta_damping, mode.delta_mac])

#cluster_meat = np.array(cluster_meat)
#Have to handle modes in the lowest order

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/strid/utils.py:723: RuntimeWarning: invalid value encountered in double_scalars
  return self.eigenvalue.imag / np.sqrt(1-self.xi**2)


In [29]:
print(cluster_meat)

[[0.0011083507891336156, 0.0006216861788345962, 0.9953043871921944], [0.0011083507891336156, 0.0006216861788345962, 0.9953043871921944], [0.0032968665085310344, 0.008755047115892277, 0.8038476715629089], [0.0032968665085310344, 0.008755047115892277, 0.8038476715629089], [0.0015979262566089823, 0.004111180694356016, 0.850676662955708], [0.0015979262566089823, 0.004111180694356016, 0.850676662955708], [0.003177044392260695, 0.000787320546791594, 0.968888037746275], [0.003177044392260695, 0.000787320546791594, 0.968888037746275], [0.009542521390606854, 0.004155380031270209, 0.990500331273891], [0.009542521390606854, 0.004155380031270209, 0.990500331273891], [0.0005284057786116761, 0.00021442754753154408, 0.9980931252914844], [0.0005284057786116761, 0.00021442754753154408, 0.9980931252914844], [0.001072533892899407, 0.0003221258589560308, 0.999847101834577], [0.001072533892899407, 0.0003221258589560308, 0.999847101834577], [0.16020829654459534, 0.42932350027783456, 0.08818590243339314], [4

### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [27]:
# https://scikit-learn.org/stable/modules/clustering.html#k-means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').